In [93]:
import urllib.request
from bs4 import BeautifulSoup
import re
import time
import sqlite3


In [94]:
def get_review(album, artist):
    con = sqlite3.connect('reviews.db')
    cur = con.cursor()
    cur.execute("SELECT * FROM reviews WHERE album=? AND artist=?", (album, artist))
    return cur.fetchone()

In [95]:
def insert_review(album, artist, rating):
    con = sqlite3.connect("reviews.db")
    cur = con.cursor()
    cur.execute("""
    INSERT INTO reviews (album, artist, rating) VALUES (?, ?, ?)
    """, (album, artist, rating))
    con.commit()

In [96]:
with urllib.request.urlopen('https://pitchfork.com/reviews/albums/') as f:
    soup = BeautifulSoup(f)

# print(soup.prettify())

In [97]:
# Get links to reviews
links = soup.find_all('a', class_='review__link')
for link in links:
    url = 'https://pitchfork.com' + link.get('href')
    with urllib.request.urlopen(url) as f:
        review = BeautifulSoup(f)
        album = review.body.main.article.header.find('h1').text
        artist = review.body.main.article.header.find('ul').text
        rating = review.body.main.article.header.find('div', class_=re.compile("^SplitScreenContentHeaderScoreBox")).text
        if get_review(album, artist) is None:
            print("Inserting: ", album, "--", artist, "--", rating)
            insert_review(album, artist, rating)
        time.sleep(5.8)